# "Intuitively understanding Python decorators"
> "Syntactic sugar and functional implementation of decorators"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, functional programming]
- hide: false
- search_exclude: true

When first utilising functional programming concepts in Python, a traditional approach is to write each function one by one, and pass the input sequentially through this pipeline. As programmers gain more experience, they tend to begin bundling all these functions in the same namespace, typically through the use of classes. However, if these classes use concepts such as inheritance, it is easy to get lost in a sea of arguments being passed around, and attempts to caress each class into the appropriate place in the pipeline. The programmer almost goes back to the original strategy of sequential transformations on input, except now they're now dealing with more complicated objects than simple functions.

Python decorators are functions that aim to counter this object-oriented problem, allowing flexible code that can dynamically modify other functions on the fly. By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

## Functions

### First-class objects
In Python, recall that functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (`string`, `int`, `float`, `list`, and so on). Consider the following three functions:

In [12]:
def cricket_runs(runs_scored:int):
    return f"You made {runs_scored} runs."

def roll_dice(number:int):
    return f"You rolled a {number}."

def general_123(specific_function):
    return specific_function("2")

Here, `cricket_runs()` and `roll_dice()` are regular functions that expect a name given as an `int`. The `general_123()` function, however, expects a function as its argument. We can, for instance, pass it the `cricket_runs()` or the `roll_dice()` function:

In [13]:
general_123(cricket_runs)

'You made 2 runs.'

In [15]:
general_123(roll_dice)

'You rolled a 2.'

Note that `general_123(roll_dice)` refers to two functions, but in different ways: `general_123()` and `roll_dice`. The `roll_dice` function is named without parentheses. This means that only a reference to the function is passed. The function is not executed. The `greet_bob()` function, on the other hand, is written with parentheses, so it will be called as usual. To get a reference to any function (i.e. to refer to as an object, rather than as something that executes), we simply don't write the parentheses:

In [16]:
general_123

<function __main__.general_123(specific_function)>

Now is a good time to recall that you can create _inner functions_ in Python (functions defined within other functions, that are only available locally), and that you can also return functions from other functions.

> Note: the main concept to grasp here is whether to return or execute a function. If you are returning a function as an _object_ to store and run later, don't use parentheses. Otherwise, if you want to execute a function on the spot, use parentheses, as per usual.

## Simple decorators
Now that you’ve seen that functions are just like any other object in Python, you’re ready to move on and see the magical beast that is the Python decorator. Let’s start with an example:

In [1]:
def standard_decorator(func):
    def wrapper():
        print("Something is happening before func is called.")
        func()
        print("Something is happening after func is called.")
    return wrapper

def shoot_ball():
    print("I shot the ball.")

shoot = standard_decorator(shoot_ball)

In [2]:
shoot()

Something is happening before func is called.
I shot the ball.
Something is happening after func is called.


The so-called decoration happens at the following line:
```python
shoot = standard_decorator(shoot_ball)
```

In effect, the name `shoot_ball` now points to the `wrapper()` inner function. Remember that you return wrapper as a function when you call `standard_decorator(shoot_ball)`:

In [4]:
shoot_ball

<function __main__.shoot_ball()>

However, `wrapper()` has a reference to the original `shoot_ball()` as `func`, and calls that function between the two calls to `print()`.

Put simply: __decorators wrap a function, modifying its behavior__.

Before moving on, let’s have a look at a second example. Because `wrapper()` is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day:

In [12]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def shoot_ball():
    print("I shot the ball!")

shoot_ball = not_during_the_night(shoot_ball)

If you try to call `say_whee()` after bedtime, nothing will happen:

In [13]:
shoot_ball()

I shot the ball!


## Syntactic sugar
The way you decorated `shoot_ball()` above is a little clunky. First of all, you end up typing the name `shoot_ball` three times. In addition, the decoration gets a bit hidden away below the definition of the function.

Instead, Python allows you to use decorators in a simpler way with the `@` symbol, sometimes called the “pie” syntax. The following example does the exact same thing as the first decorator example:

In [15]:
def standard_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@standard_decorator
def shoot_ball():
    print("I shot the ball!")

So, `@standard_decorator` is just an easier way of saying `shoot_ball = standard_decorator(shoot_ball)`. It’s how you apply a decorator to a function.

### Reusing decorators
Recall that a decorator is just a regular Python function. All the usual tools for easy reusability are available. Let’s move the decorator to its own module that can be used in many other functions.

Create a file called `decorators.py` with the following content:

In [27]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

> Note: You can name your inner function whatever you want, and a generic name like `wrapper()` is usually okay. You’ll see a lot of decorators in this article. To keep them apart, we’ll name the inner function with the same name as the decorator but with a `wrapper_ prefix`.

You can now use this new decorator in other files by doing a regular import:

```python
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")
```

When you run this example, you should see that the original say_whee() is executed twice:

```python
>>> say_whee()
Whee!
Whee!
```

### Decorating functions with arguments
Say that you have a function that accepts some arguments. Can you still decorate it? Let’s try:

In [28]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

Unfortunately, running this code raises an error:

In [29]:
# collapse
greet("World")

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

The problem is that the inner function `wrapper_do_twice()` does not take any arguments, but `name="World"` was passed to it. You could fix this by letting `wrapper_do_twice()` accept one argument, but then it would not work for the `say_whee()` function you created earlier.

The solution is to use `*args` and `**kwargs` in the inner wrapper function. Then it will accept an arbitrary number of positional and keyword arguments. Rewrite `do_twice()` as follows:

In [30]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

The `wrapper_do_twice()` inner function now accepts any number of arguments and passes them on to the function it decorates. Now both your `say_whee()` and `greet()` examples works:

In [31]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [32]:
@do_twice
def greet(name):
    print(f"Hello {name}")

greet("World")

Hello World
Hello World


### Returning values from decorated functions
What happens to the return value of decorated functions? Well, that’s up to the decorator to decide. Let’s say you decorate a simple function as follows:

In [33]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

In [34]:
hi_adam = return_greeting("Adam")

Creating greeting
Creating greeting


In [35]:
print(hi_adam)

None


Oops, your decorator ate the return value from the function.

Because the `do_twice_wrapper()` doesn’t explicitly return a value, the call `return_greeting("Adam")` ended up returning `None`.

To fix this, you need to make sure the wrapper function returns the return value of the decorated function. Change your decorator:

In [40]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

The return value from the last execution of the function is returned:

In [42]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

return_greeting("Adam")

Creating greeting
Creating greeting


'Hi Adam'

## Introspection
A great convenience when working with Python, especially in the interactive shell, is its powerful introspection ability. Introspection is the ability of an object to know about its own attributes at runtime. For instance, a function knows its own name and documentation:

In [64]:
print

<function print>

In [68]:
print.__name__

'print'

In [69]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



The introspection works for functions you define yourself as well:

In [70]:
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

In [71]:
say_whee.__name__

'wrapper'

However, after being decorated, `say_whee()` has gotten very confused about its identity. It now reports being the `wrapper_do_twice()` inner function inside the `do_twice()` decorator. Although technically true, this is not very useful information.

To fix this, decorators should use the `@functools.wraps` decorator, which will preserve information about the original function. Update decorator again:

In [72]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

You do not need to change anything about the decorated `say_whee()` function:

In [76]:
@do_twice
def say_whee():
    print("Whee!")

say_whee

<function __main__.say_whee()>

In [77]:
say_whee.__name__

'say_whee'

Much better! Now `say_whee()` is still itself after decoration.

## Some examples
Let’s look at a few more useful examples of decorators. You’ll notice that they’ll mainly follow the same pattern that you’ve learned so far:

```python
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator
```

This formula is a good boilerplate template for building more complex decorators.

### Timing decorator
Let’s start by creating a `@timer` decorator. It will measure the time a function takes to execute and print the duration to the console. Here’s the code:

In [44]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    
        value = func(*args, **kwargs)
        end_time = time.perf_counter()      
        run_time = end_time - start_time    
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

This decorator works by storing the time just before the function starts running (at the line marked # 1) and just after the function finishes (at # 2). The time the function takes is then the difference between the two (at # 3). We use the `time.perf_counter()` function, which does a good job of measuring time intervals. Here are some examples of timings:

In [45]:
waste_some_time(1)

Finished 'waste_some_time' in 0.0107 secs


In [46]:
waste_some_time(999)

Finished 'waste_some_time' in 2.3977 secs


> Note: The `@timer` decorator is great if you just want to get an idea about the runtime of your functions. If you want to do more precise measurements of code, you should instead consider the `timeit` module in the standard library. It temporarily disables garbage collection and runs multiple trials to strip out noise from quick function calls.

## Fancy decorators

### Decorating classes
There are two different ways you can use decorators on classes. The first one is very close to what you have already done with functions: you can decorate the methods of a class. This was one of the motivations for introducing decorators back in the day.

Some commonly used decorators that are even built-ins in Python are `@classmethod`, `@staticmethod`, and `@property`. The `@classmethod` and `@staticmethod` decorators are used to define methods inside a class namespace that are not connected to a particular instance of that class. The `@property` decorator is used to customise getters and setters for class attributes.

The following definition of a `Circle` class uses the `@classmethod`, `@staticmethod`, and `@property` decorators:

In [47]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In this class:

* `cylinder_volume()` is a regular method.
* `radius` is a mutable property: it can be set to a different value. However, by defining a setter method, we can do some error testing to make sure it’s not set to a nonsensical negative number. Properties are accessed as attributes without parentheses.
* `area` is an immutable property: properties without `.setter()` methods can’t be changed. Even though it is defined as a method, it can be retrieved as an attribute without parentheses.
* `unit_circle()` is a class method. It’s not bound to one particular instance of `Circle`. Class methods are often used as factory methods that can create specific instances of the class.
* `pi()` is a static method. It’s not really dependent on the `Circle` class, except that it is part of its namespace. Static methods can be called on either an instance or the class.

The `Circle` class can for example be used as follows:

In [48]:
c = Circle(5)
c.radius

5

In [49]:
c.area

78.5398163375

In [50]:
c.radius = 2 
c.area

12.566370614

In [51]:
c.area = 100

AttributeError: can't set attribute

In [52]:
c.cylinder_volume(height=4)

50.265482456

In [53]:
c.radius = -1

ValueError: Radius must be positive

In [55]:
c.pi()

3.1415926535

In [56]:
Circle.pi()

3.1415926535

Let’s define a class where we decorate some of its methods using the `@debug` and `@timer` decorators from earlier:

In [61]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug

class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

Using this class, you can see the effect of the decorators:

In [62]:
tw = TimeWaster(1000)

Calling __init__(<__main__.TimeWaster object at 0x7fdd48667d00>, 1000)
'__init__' returned None


In [63]:
tw.waste_time(999)

Finished 'waste_time' in 0.2636 secs


### Decorators with arguments
Sometimes, it’s useful to pass arguments to your decorators. For instance, `@do_twice` could be extended to a `@repeat(num_times)` decorator. The number of times to execute the decorated function could then be given as an argument.

Think about how you could achieve this. So far, the name written after the @ has referred to a function object that can be called with another function. To be consistent, you then need repeat(num_times=4) to return a function object that can act as a decorator. Luckily, you already know how to return functions!

Typically, the decorator creates and returns an inner wrapper function, so writing the example out in full will give you an inner function within an inner function. While this might sound like the programming equivalent of the Inception movie, we’ll untangle it all in a moment:

In [58]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

It looks a little messy, but we have only put the same decorator pattern you have seen many times by now inside one additional `def` that handles the arguments to the decorator. Let’s start with the innermost function:

```python
def wrapper_repeat(*args, **kwargs):
    for _ in range(num_times):
        value = func(*args, **kwargs)
    return value
```

This `wrapper_repeat()` function takes arbitrary arguments and returns the value of the decorated function, `func()`. This wrapper function also contains the loop that calls the decorated function `num_times` times. This is no different from the earlier wrapper functions you have seen, except that it is using the `num_times` parameter that must be supplied from the outside.

One step out, you’ll find the decorator function:

```python
def decorator_repeat(func):
    @functools.wraps(func)
    def wrapper_repeat(*args, **kwargs):
        ...
    return wrapper_repeat
```

Again, `decorator_repeat()` looks exactly like the decorator functions you have written earlier, except that it’s named differently. That’s because we reserve the base name—`repeat()`—for the outermost function, which is the one the user will call.

As you have already seen, the outermost function returns a reference to the decorator function:

```python
def repeat(num_times):
    def decorator_repeat(func):
        ...
    return decorator_repeat
```

There are a few subtle things happening in the repeat() function:

Defining `decorator_repeat()` as an inner function means that `repeat()` will refer to a function object—`decorator_repeat`. Earlier, we used repeat without parentheses to refer to the function object. The added parentheses are necessary when defining decorators that take arguments.

The `num_times` argument is seemingly not used in `repeat()` itself. But by passing `num_times` a closure is created where the value of `num_times` is stored until it will be used later by `wrapper_repeat()`.

With everything set up, let’s see if the results are as expected:

In [60]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")
    
greet("World")

Hello World
Hello World
Hello World
Hello World
